In [3]:
!pip install psycopg2-binary

    100% |████████████████████████████████| 1.5MB 7.9MB/s 


In [4]:
import psycopg2 as pg2
from psycopg2.extras import RealDictCursor, Json
import json
import pandas as pd

In [5]:
%run sql_test.py

In [6]:
IP_ADDRESS = '34.222.246.54'
DBNAME = 'postgres'
USER = 'postgres'
PASSWORD = 'foobar1'

In [7]:
def con_cur_to_db(dbname=DBNAME, dict_cur=None):
    ''' 
    Returns both a connection and a cursor object for your database
    '''

    con = pg2.connect(host=IP_ADDRESS, #allows you to navigate db
                  dbname=dbname,
                  user=USER,
                  password=PASSWORD)
    if dict_cur:
        cur = con.cursor(cursor_factory=RealDictCursor)
    else:
        cur = con.cursor()
    return con, cur
    
def execute_query(query, dbname=DBNAME, dict_cur=None, command=False):
    '''
    Executes a query directly to a database, without having to create a cursor and connection each time. 
    '''
    con, cur = con_cur_to_db(dbname, dict_cur)
    cur.execute(f'{query}')
    if not command:
        data = cur.fetchall()
        con.close()
        return data
    con.commit() #sends to server
    con.close() #closes server connection

In [8]:
def insert_entry_json(data, tablename=None):
    con, cur = con_cur_to_db()
    for x in data:
        cur.execute(f'INSERT INTO {tablename} (data) VALUES ({Json(x)});')
    con.commit()
    con.close()

query = '''CREATE TABLE raw_tweets
(id SERIAL,
data JSONB);'''

query = '''DROP TABLE flex_test;'''

execute_query(query, command=True)

with open('./tweets_ex.json', 'r') as f:
    raw_data = json.load(f)

insert_entry_json(raw_data, raw_tweets)

query = '''SELECT * FROM flex_test;'''

response = execute_query(query, dict_cur=True)

response[0]['data']['text']

execute_query(query)

query = """SELECT data->>'text'
FROM flex_test
WHERE data->>'text' IS LIKE 'RT';
"""

response = execute_query(query, dict_cur=True)

In [ ]:
len(response)

In [45]:
df = pd.DataFrame(response)

In [46]:
df.head()

,?column?
0,RT @MitzProductions: I’ve been to a shit load ...
1,Hoje achei nos meus arquivos o áudio de Kind o...
2,47. El show del mediotiempo del Superbowl de e...
3,RT @happilyabby: rt if you stan:\n\n- one dire...
4,na mrl?\njustin timberlake - mirrors


In [13]:
query = """SELECT data#>'{place,bounding_box,coordinates}'
FROM raw_tweets limit 5000;
"""

In [14]:
response = execute_query(query, dict_cur=True)

df_geo = pd.DataFrame(response).dropna()

In [15]:
for tweet in df_geo['?column?']:
    inside = tweet[0][1]
    outside = tweet[0][3]
    df_geo['lat'] = (inside[0] + outside[0])/2
    df_geo['long'] = (inside[1] + outside[1])/2

In [16]:
df_geo

,?column?,lat,long
2,"[[[-81.046876, 35.001706], [-80.646695, 35.001...",-80.846785,35.209059
14,"[[[-81.046876, 35.001706], [-80.646695, 35.001...",-80.846785,35.209059
26,"[[[-81.046876, 35.001706], [-80.646695, 35.001...",-80.846785,35.209059
38,"[[[-81.046876, 35.001706], [-80.646695, 35.001...",-80.846785,35.209059
50,"[[[-81.046876, 35.001706], [-80.646695, 35.001...",-80.846785,35.209059
62,"[[[-81.046876, 35.001706], [-80.646695, 35.001...",-80.846785,35.209059
171,"[[[-81.046876, 35.001706], [-80.646695, 35.001...",-80.846785,35.209059
185,"[[[-81.046876, 35.001706], [-80.646695, 35.001...",-80.846785,35.209059
199,"[[[-81.046876, 35.001706], [-80.646695, 35.001...",-80.846785,35.209059
213,"[[[-81.046876, 35.001706], [-80.646695, 35.001...",-80.846785,35.209059
